In [1]:
from pulp import *

# Declaring variables
f1 = LpVariable("f1", 0, upBound=1, cat='Integer')
f2 = LpVariable("f2", 0, upBound=1, cat='Integer')
f3 = LpVariable("f3", 0, upBound=1, cat='Integer')
f4 = LpVariable("f4", 0, upBound=1, cat='Integer')
f5 = LpVariable("f5", 0, upBound=1, cat='Integer')
f6 = LpVariable("f6", 0, upBound=1, cat='Integer')
f7 = LpVariable("f7", 0, upBound=1, cat='Integer')
f8 = LpVariable("f8", 0, upBound=1, cat='Integer')

backlog = [
    { 'feature': f1, 'frontend_dev': 5, 'backend_dev': 3, 'design': 2, 'points': 3 },
    { 'feature': f2, 'frontend_dev': 10, 'backend_dev': 10, 'design': 4, 'points': 8 },
    { 'feature': f3, 'frontend_dev': 0, 'backend_dev': 25, 'design': 0, 'points': 13 },
    { 'feature': f4, 'frontend_dev': 18, 'backend_dev': 6, 'design': 3, 'points': 5 },
    { 'feature': f5, 'frontend_dev': 8, 'backend_dev': 6, 'design': 12, 'points': 1 },
    { 'feature': f6, 'frontend_dev': 10, 'backend_dev': 4, 'design': 8, 'points': 21 },
    { 'feature': f7, 'frontend_dev': 2, 'backend_dev': 10, 'design': 4, 'points': 13 },
    { 'feature': f8, 'frontend_dev': 4, 'backend_dev': 6, 'design': 2, 'points': 2 }
]

team_availabilities = { 
    'frontend_dev': 40,
    'backend_dev': 35,
    'design': 40
}

feature_names = [feature['feature'] for feature in backlog]


# Defining the problem
prob = LpProblem("Sprint Planning", LpMaximize)

# Adding constraings
for dev, availability in team_availabilities.items():
    feature_requirements_for_dev = [feature[dev] for feature in backlog]
    prob += sum([feature * hours for feature, hours in zip(feature_names, feature_requirements_for_dev)]) <= availability


# Defining the objective function
story_points = [feature['points'] for feature in backlog]
prob += sum([feature * points for feature, points in zip(feature_names, story_points)])

# Solving the problem
status = prob.solve(GLPK(msg=0))
LpStatus[status]

prob


Sprint Planning:
MAXIMIZE
3*f1 + 8*f2 + 13*f3 + 5*f4 + 1*f5 + 21*f6 + 13*f7 + 2*f8 + 0
SUBJECT TO
_C1: 5 f1 + 10 f2 + 18 f4 + 8 f5 + 10 f6 + 2 f7 + 4 f8 <= 40

_C2: 3 f1 + 10 f2 + 25 f3 + 6 f4 + 6 f5 + 4 f6 + 10 f7 + 6 f8 <= 35

_C3: 2 f1 + 4 f2 + 3 f4 + 12 f5 + 8 f6 + 4 f7 + 2 f8 <= 40

VARIABLES
0 <= f1 <= 1 Integer
0 <= f2 <= 1 Integer
0 <= f3 <= 1 Integer
0 <= f4 <= 1 Integer
0 <= f5 <= 1 Integer
0 <= f6 <= 1 Integer
0 <= f7 <= 1 Integer
0 <= f8 <= 1 Integer

In [2]:
pulp.value(prob.objective)

47

In [3]:
import pandas as pd
implement_flags = []

for v in prob.variables():
	implement_flags.append(v.varValue)
    
    
frontend_hours = [feature['frontend_dev'] for feature in backlog]
backend_hours = [feature['backend_dev'] for feature in backlog]
design_hours = [feature['design'] for feature in backlog]

frontend_time = sum([feature * hours for feature, hours in zip(frontend_hours, implement_flags)])
df = pd.DataFrame({'Feature': feature_names, 'Story Points': story_points, 'Implement': implement_flags, 'Frontend': frontend_hours, 'Backend': backend_hours, 'Design': design_hours })
df.index = df["Feature"]
weekly_plan = df[df["Implement"] == 1]

weekly_plan

,Backend,Design,Feature,Frontend,Implement,Story Points
Feature,,,,,,
f2,10,4,f2,10,1,8
f4,6,3,f4,18,1,5
f6,4,8,f6,10,1,21
f7,10,4,f7,2,1,13


In [4]:
total = pd.DataFrame([['-',sum(weekly_plan['Backend']), sum(weekly_plan['Design']), sum(weekly_plan['Frontend']), sum(weekly_plan['Story Points'])]], columns=list(['Feature','Backend', 'Design', 'Frontend', 'Story Points']))
total.index = ['Total']
weekly_plan = weekly_plan.append(total)

In [5]:
cols = ['Feature', 'Story Points', 'Frontend', 'Backend', 'Design']
weekly_plan = weekly_plan[cols]
weekly_plan

,Feature,Story Points,Frontend,Backend,Design
f2,f2,8,10,10,4
f4,f4,5,18,6,3
f6,f6,21,10,4,8
f7,f7,13,2,10,4
Total,-,47,40,30,19


In [17]:
d = pd.DataFrame(backlog)
d

,backend_dev,design,feature,frontend_dev,points
0,3,2,f1,5,3
1,10,4,f2,10,8
2,25,0,f3,0,13
3,6,3,f4,18,5
4,6,12,f5,8,1
5,4,8,f6,10,21
6,10,4,f7,2,13
7,6,2,f8,4,2


In [20]:
d = d[['feature', 'points', 'frontend_dev', 'backend_dev', 'design' ]]
d

,feature,points,frontend_dev,backend_dev,design
0,f1,3,5,3,2
1,f2,8,10,10,4
2,f3,13,0,25,0
3,f4,5,18,6,3
4,f5,1,8,6,12
5,f6,21,10,4,8
6,f7,13,2,10,4
7,f8,2,4,6,2


In [21]:
d.to_csv


<bound method DataFrame.to_csv of   feature  points  frontend_dev  backend_dev  design
0      f1       3             5            3       2
1      f2       8            10           10       4
2      f3      13             0           25       0
3      f4       5            18            6       3
4      f5       1             8            6      12
5      f6      21            10            4       8
6      f7      13             2           10       4
7      f8       2             4            6       2>

In [22]:
dev = pd.DataFrame(team_availabilities)

ValueError: If using all scalar values, you must pass an index

In [23]:
team_availabilities


{'backend_dev': 35, 'design': 40, 'frontend_dev': 40}

In [24]:
weekly_plan.to_csv

<bound method DataFrame.to_csv of       Feature  Story Points  Frontend  Backend  Design
f2         f2             8        10       10       4
f4         f4             5        18        6       3
f6         f6            21        10        4       8
f7         f7            13         2       10       4
Total       -            47        40       30      19>